# 数理統計学 9節解答

## 使うモジュールのインポート
この節では、`scipy.stats.norm, scipy.stats.t, scipy.stats.chi2`を使う。  
それぞれが、正規分布、t分布、カイ自乗分布に対応している。  
`scipy.stats.norm.interval(alpha, loc, scale)`は$\mathcal{N}\left(\text{loc}, \text{scale}\right)$の両側$\alpha$点を出力する。  
詳細については、[stats.norm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html), [stats.t](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html), [stats.chi2](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2.html)を参照。

In [1]:
import numpy as np
from scipy import stats

## 9.1
### 正規分布の平均の区間推定 (1), (2)
**分散が既知のとき**  
分散が既知のとき、$1-\alpha$の信頼区間は$$\left[\bar{X}-z_{\alpha}^{*}\frac{\sigma}{\sqrt{n}},\ \bar{X}+z_{\alpha}^{*}\frac{\sigma}{\sqrt{n}}\right]$$
である(p.156)。  

**分散が未知のとき**  
分散が未知のとき、$1-\alpha$の信頼区間は$$\left[\bar{X}-t_{n-1,\alpha}^{*}\frac{\hat{\sigma}}{\sqrt{n}},\ \bar{X}+t_{n-1,\alpha}^{*}\frac{\hat{\sigma}}{\sqrt{n}}\right]$$
である(p157)。（分散が既知のときとの違いは、分散を不偏分散で置き換え、正規分布の両側$\alpha$点を自由度$n-1$t分布の両側$\alpha$点で置き換えたことだけである。）  

**注意**  
実は、$z_{\alpha}^{*}\frac{\sigma}{\sqrt{n}}$は標準正規分布を$\frac{\sigma}{\sqrt{n}}$でスケーリングし直した分布の両側$\alpha$点になっている。   
（正規分布の定数倍は、平均をその定数倍、分散を定数の2乗倍することに対応する。）  
つまり、$\mathcal{N}\left(\bar{X}, \frac{\sigma^2}{n}\right)$の両側$\alpha$点を求めれば、それが信頼区間に対応している。  
分散が未知のときも、t分布に対して同様のことが言える。  
そこで、今回は計算機の強みを活かして、標準正規分布の両側$\alpha$点($z_{\alpha}^{*}$)を使わずに、直接$\mathcal{N}\left(\bar{X}, \frac{\sigma^2}{n}\right)$の両側$\alpha$点を見つける。  
それが、`stats.norm.interval(q, X_bar, np.sqrt(sigma2/n)`である。

### 正規分布の分散の区間推定
分散の$1-\alpha$の信頼区間は$$\left[\frac{(n-1)\hat{\sigma}^2}{\chi_{n-1,\frac{\alpha}{2}}^2}, \frac{(n-1)\hat{\sigma}^2}{\chi_{n-1,1-\frac{\alpha}{2}}^2} \right]$$
である(p159)。

In [2]:
# 9.1
samples = [12.7, 6.6, 5.6, 14.3, 11.4, 10.8, 13.8, 11.2, 10.0, 12.8, 7.1, 14.0]
n = len(samples) # sample size
q = 0.95 # confidence level
X_bar = np.mean(samples) # mean
sigma2 = 7 # known variance
s2 = np.var(samples, ddof=1) # unbiased variance

# (1)
print('Answer of 9.1 (1) is:', stats.norm.interval(q, X_bar, np.sqrt(sigma2/n)))

# (2)
print('Answer of 9.1 (2) is:', stats.t.interval(q, n-1, X_bar, np.sqrt(s2/n)))

# (3)
chi_left = stats.chi2.ppf((1-q)/2, n-1) # ppfは分布関数(cdf)の逆関数
chi_right = stats.chi2.ppf(q+(1-q)/2, n-1)
print('Answer of 9.1 (3) is:', ((n-1)*s2/chi_right, (n-1)*s2/chi_left))

Answer of 9.1 (1) is: (9.361386113576325, 12.355280553090344)
Answer of 9.1 (2) is: (8.956183398904026, 12.760483267762643)
Answer of 9.1 (3) is: (4.497670853412747, 25.837440037846203)


## 9.2
p159下部のベルヌーイ分布における$1-\alpha$信頼区間を求める式を実装する。(`bernouli_interval`)  
この関数は、標本平均、1-信頼水準、標本数を受け取ったときに、信頼区間を返す。

In [3]:
# 9.2
def bernouli_interval(X_bar, q, n):
    '''
    input: 
        X_bar: avarage of samples
        q: 1-q is confidence level
        n: sample size
    output:
        confidence interval of bernouli distribution (p159 in Inagaki's textbook)
    '''
    z_a = stats.norm.interval(q)[1]
    first_item = X_bar+z_a**2/(2*n)
    second_item = z_a/np.sqrt(n)*np.sqrt(X_bar*(1-X_bar)+z_a**2/(4*n))
    return np.array([first_item-second_item, first_item+second_item]/(1+z_a**2/n))


n = 300
q = 0.95
X_bar = 180/300
print('Answer of 9.2 is:', bernouli_interval(X_bar, q, n))

Answer of 9.2 is: [0.5436366  0.65383481]


## 9.3
(1)と(2)は9.1と同じ。(3)は2つの正規分布の平均差の区間推定が必要。

### 2つの正規分布の平均差の区間推定
**分散が既知のとき**  
分散が既知のとき、$1-\alpha$の信頼区間は$$\left[\bar{X}-\bar{Y}-z_{\alpha}^{*}\sqrt{\frac{\sigma_1^2}{m}+\frac{\sigma_2^2}{n}},\ \bar{X}-\bar{Y}+z_{\alpha}^{*}\sqrt{\frac{\sigma_1^2}{m}+\frac{\sigma_2^2}{n}}\right]$$
である(p.156)。  

**分散が未知のとき**  
分散が未知のとき、$1-\alpha$の信頼区間は$$\left[\bar{X}-\bar{Y}-t_{m+n-2,\alpha}^{*}\hat{\sigma}\sqrt{\frac{1}{m}+\frac{1}{n}},\ \bar{X}-\bar{Y}+t_{m+n-2,\alpha}^{*}\hat{\sigma}\sqrt{\frac{1}{m}+\frac{1}{n}}\right]$$
である(p157)。（分散が等しいという仮定が必要）

In [4]:
# 9.3
sample_A = [1293, 1385, 1614, 1497, 1340, 1643, 1466, 1094, 1270, 1028, 1711, 1627]
sample_B = [1061, 1065, 1383, 1090, 1021, 1138, 1070, 1143]
q = 0.95
n_A = len(sample_A)
n_B = len(sample_B)
X_barA = np.mean(sample_A)
X_barB = np.mean(sample_B)
s2_A = np.var(sample_A, ddof=1)
s2_B = np.var(sample_B, ddof=1)

# (1)
print('Answer of 9.3 (1) mu_1 is:', stats.t.interval(q, n_A-1, X_barA, np.sqrt(s2_A/n_A)))
print('Answer of 9.3 (1) mu_2 is:', stats.t.interval(q, n_B-1, X_barB, np.sqrt(s2_B/n_B)))

# (2)
chi_leftA = stats.chi2.ppf((1-q)/2, n_A-1) # ppfは分布関数(cdf)の逆関数
chi_rightA = stats.chi2.ppf(q+(1-q)/2, n_A-1)
chi_leftB = stats.chi2.ppf((1-q)/2, n_B-1) # ppfは分布関数(cdf)の逆関数
chi_rightB = stats.chi2.ppf(q+(1-q)/2, n_B-1)
print('Answer of 9.3 (2) sigma_1 is:', ((n_A-1)*s2_A/chi_rightA, (n_A-1)*s2_A/chi_leftA))
print('Answer of 9.3 (2) sigma_2 is:', ((n_B-1)*s2_B/chi_rightB, (n_B-1)*s2_B/chi_leftB))

# (3)
X_diff = X_barA - X_barB
s2_diff = (s2_A*(n_A-1) + s2_B*(n_B-1))/(n_A+n_B-2)
print('Answer of 9.3 (3) is:', stats.t.interval(q, n_A+n_B-2, X_diff, np.sqrt(s2_diff)*np.sqrt(1/n_A+1/n_B)))

Answer of 9.3 (1) mu_1 is: (1274.7049686545536, 1553.2950313454464)
Answer of 9.3 (1) mu_2 is: (1026.7299437799406, 1216.0200562200594)
Answer of 9.3 (2) sigma_1 is: (24119.56258420238, 138557.88302862243)
Answer of 9.3 (2) sigma_2 is: (5602.6475792279625, 53089.24266199098)
Answer of 9.3 (3) is: (114.8816420259761, 470.36835797402387)


## 9.4
差のサンプルを正規分布で近似しているので、とりあえず差のデータを作る。（`samples`)  
このsamplesの区間推定は9.1 (2)と同様。

In [6]:
# 9.4
samples = [65-74, 74-80, 71-86, 73-95, 74-92, 68-98, 75-74, 65-77, 68-89, 69-87, 67-95, 70-97, 71-85, 70-83, 74-73]
q = 0.95
n = len(samples)
X_bar = np.mean(samples)
s2 = np.var(samples, ddof=1)
print('Answer of 9.4 is:', stats.t.interval(q, n-1, X_bar, np.sqrt(s2/n)))

Answer of 9.4 is: (-20.698472778537997, -10.101527221462003)


## 9.5
9.4のデータについて、9.3 (3)と同じ方法で「2つの正規分布の平均差の区間推定」を行う。  
結果を見ると、9.4のときよりも信頼区間は狭い。

In [7]:
# 9.5
samples_after = [65, 74, 71, 73, 74, 68, 75, 65, 68, 69, 67, 70, 71, 70, 74]
samples_pre = [74, 80, 86, 95, 92, 98, 74, 77, 89, 87, 95, 97, 85, 83, 73]
q = 0.95
n_pre = len(samples_pre)
n_after = len(samples_after)
X_pre = np.mean(samples_pre)
X_after = np.mean(samples_after)
s2_pre = np.var(samples_pre, ddof=1)
s2_after = np.var(samples_after, ddof=1)
X_diff = X_after-X_pre
s2_diff = (s2_pre*(n_pre-1) + s2_after*(n_after-1))/(n_pre+n_after-2)
print('Answer of 9.5 is:', stats.t.interval(q, n_pre+n_after-2, X_diff, np.sqrt(s2_diff)*np.sqrt(2/n_pre)))

Answer of 9.5 is: (-20.309398634009472, -10.490601365990539)


## 9.6
p164の2つの正規分布の分散比の区間推定を行う。

In [8]:
# 9.6
q = 0.90
r_hat = s2_pre/s2_after
f_left = stats.f.ppf((1-q)/2, n_pre-1, n_after-1)
f_right = stats.f.ppf(q+(1-q)/2, n_pre-1, n_after-1)
print('Answer of 9.6 is:', (r_hat/f_right, r_hat/f_left))

Answer of 9.6 is: (2.8151455509691474, 17.366333251793176)


## 9.7
p164の2つの比率の差の区間推定を行う。

In [9]:
# 9.7
q = 0.95
X_man = 55/100
X_wo = 48/60
X_diff = X_man-X_wo
n_man = 100
n_wo = 60
q = 0.95
print('Answer for 9.7 is:', stats.norm.interval(q, X_diff, np.sqrt(X_man*(1-X_man)/n_man + X_wo*(1-X_wo)/n_wo)))

Answer for 9.7 is: (-0.39054003262321957, -0.1094599673767804)


## 9.8
9.2と9.7の融合問題

In [10]:
# 9.8
q =  0.95
X_1 = 160/220
X_2 = 160/200
n_1 = 220
n_2 = 200
print('The first answer is:', bernouli_interval(X_1, q, n_1)) # bernouli_interval is defined in #9.2
print('The second answer is:', bernouli_interval(X_2, q, n_2))

X_diff = X_1-X_2
print('The third answer is:', stats.norm.interval(q, X_diff, np.sqrt(X_1*(1-X_1)/n_1 + X_2*(1-X_2)/n_2)))
print('2つの平均の差の95%信頼区間は0を含んでいるため、この結果からは発芽率に差があるとは言えない。')

The first answer is: [0.66489889 0.78184588]
The second answer is: [0.73914481 0.84954799]
The third answer is: (-0.153576149648384, 0.00812160419383852)
2つの平均の差の95%信頼区間は0を含んでいるため、この結果からは発芽率に差があるとは言えない。
